# Vectorización de texto
### Creación de los dataframe y limpieza de texto

In [2]:
import pandas as pd
import numpy as np
from pprint import pprint

df = pd.read_csv('pericias_medicas.csv', sep=',',  encoding='utf-8')
#convierte en string los datos de la columna text
df1 = df['text'].apply(str)

# Fase de limpieza

In [4]:
#Limpieza de caracteres no deseados
import nltk
import re

def general(txt: str, bert=False, nums=False) -> str:
    """
    Elimina caracteres no deseados
    Params:
        **txt**:texto a ser limpiado de caracteres no desaeados
    """
    if nums:
        txt = re.sub(r'\d+', ' ', txt)
    if not bert:
        txt = txt.translate(str.maketrans(
            'áéíóúýàèìòùÁÉÍÓÚÀÈÌÒÙÝ', 'aeiouyaeiouAEIOUAEIOUY'))
        txt = re.sub(r'[^\w\s:.)-]', '', txt)

    txt = txt.replace('\r', ' ').replace("\v", ' ').replace(
        "\t", ' ').replace("\f", ' ').replace("\a", ' ').replace("\b", ' ')
    txt = re.sub(' +', ' ', txt)
    txt = txt.strip()
    return txt

dfLimpio = df1.apply(general)


In [5]:
#limpieza de stopwords

stops = nltk.corpus.stopwords.words('spanish')

def remove_stops(texto: str) -> str:
    """
    Función que elimina stopwords
    Params:
        **texto**:texto a ser limpiado de stopwords

    """
    texto = [
        i for i in texto.split() if i not in stops
    ]
    return ' '.join(texto)

dfLimpio = dfLimpio.apply(remove_stops)

In [6]:
dfLimpio[0]

'Informa continuacion: J.T. 13 PERITO MEDICO PRESENTA INFORME PERICIAL Autos: BOIERO FRANCO IVAN cSWISS MEDICAL ART S.A. sAccidente - Ley Especial Expediente N.º: 713992017 I-PROEMIO: Sr. JUEZ NACIONAL: HORTAS MARIA ANDREA S. perito medico oficio siguientes autos domicilio constituido calle Austria 1754 3º piso Dpto. 10 Capital Z 172 tel. 4325-8802 domicilio electronico mashortashotmail.com 27- 16523394-3 cumplimiento dispuesto V.S. referente presentes autos despues haber examinado dijo ser Boiero Franco Ivan tomaron momento examen clinico actual consideraron capacidad generica particular puntos pericia solicitados partes V.S.: II-ANTECEDENTES DE INTERES MEDICO-LEGAL: Foja 3 Inicia demanda Foja 6 vta. Hechos accidente in itinere refiere 20 setiembre 2017 siendo 5:10h encontraba camino trabajo cruzar calle piso piedra pie izquierdo queda trabado girando cuerpo sufriendo entorsis tobillo izquierdo asi consecuencia fuerte dolor da aviso empleador realiza correspondiente denuncia ART reali

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

frequency_vectorizer = CountVectorizer(analyzer=dfLimpio[0])

frequency_vectorizer.fit(df1[0])
frequency_vectors = frequency_vectorizer.transform(df1[0])

ValueError: Iterable over raw text documents expected, string object received.

# TF-IDF